In [1]:
class CFG:
    
    USE_GPU= False
    test =0.2
    normalize = False#True
    overSampling = True # handling imbalance class
    crossValidate = False#True # Support cross Validation 
    nFold = 5#4 # 8  No remainder  for oversampling 


    USE_XGB = True
    USE_TABNET = True

    # XGBoost
    xgbEstimate = 2 # 250 , 300 
    xgbDepth = 7 #5 #6, low value avoid overfit
    xgbLR = 0.05 # 0.08

    # Tabnet model hyperparameter 
    maxEpochs = 1000
    n_d =32 #8
    n_a = 32 # 8
    n_steps = 5 # 3
    mask_type = "entmax" # "sparsemax"
    learningRate = 1e-3 # 1e-2
    
    trainSeriesParquet = "series_train.parquet"
    testSeriesParquet = "series_test.parquet"
    # rootDir ="/kaggle/input/child-mind-institute-problematic-internet-use/"
    rootDir = "/Users/jiazhiyuan/personal_repos/child-health/data/"
    dataDictionary = rootDir+"data_dictionary.csv"
    trainDataFile= rootDir+"train.csv"
    testDataFile = rootDir+"test.csv"
    samplesub = rootDir+"sample_submission.csv"
    trainSerise1Path = rootDir+"series_train.parquet/id=00115b9f/part-0.parquet"
    pretrained_weight = rootDir+"model.xgb"

import os, gc, time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from IPython.display import Markdown
import ctypes
import seaborn as sns
from tqdm import tqdm

# import torch
# from pytorch_tabnet.tab_model import TabNetClassifier


from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import (train_test_split, KFold, GridSearchCV, StratifiedKFold,
                                    cross_val_score)
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# over Sample 
from imblearn.over_sampling import SMOTE

from sklearn.metrics import ( mean_absolute_error, 
                             mean_squared_error, 
                             r2_score, 
                             cohen_kappa_score)

from sklearn.model_selection import (train_test_split, 
                                     KFold, 
                                     GridSearchCV, 
                                     StratifiedKFold)


# import torch
# ML library 
from xgboost import XGBRFRegressor, XGBClassifier# plot_importance, plot_tree
import xgboost
from lightgbm import LGBMRegressor, LGBMClassifier
import lightgbm
from catboost import CatBoostRegressor, CatBoostClassifier
import catboost
from sklearn.ensemble import (RandomForestRegressor,  RandomForestClassifier ,
                              VotingRegressor, VotingClassifier )

# multiple processing lib
from joblib import Parallel , delayed
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")    


In [2]:
trainSerise1 = pd.read_parquet(CFG.trainSerise1Path)
trainSerise1.drop("step", axis=1, inplace=True)
sensorFeature =trainSerise1.columns.to_list()
statFeat = trainSerise1.describe().index.to_list() 
sensorStatCol = []
for sta in statFeat: # start statist row 
    # print(sta)
    for item in sensorFeature:
        # print(item)
        sensorStatCol.append(f"{item}_{sta}")
trainDataDF = pd.read_csv(CFG.trainDataFile)
testDataDF = pd.read_csv(CFG.testDataFile)
submit = pd.read_csv(CFG.samplesub)
# %%time

def loadParquetFile(directory, fileName):
    """
    read parquet file 
    """
    path = os.path.join(directory, fileName, "part-0.parquet")
    df = pd.read_parquet(path)
    df.drop("step", axis=1, inplace=True) # drop step column
    statDF = df.describe().values.reshape(-1)
    return statDF, fileName.split("=")[1] # get ids
    

def loadTimeSeriesData(directory):
    """
    input : root direction
    """
#     print("DIR :", directory)
    filesIds = os.listdir(directory) # get list of folder name (files ids)
#     print(filesIds)
    with ThreadPoolExecutor() as executor:
#     with ProcessPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: loadParquetFile(directory, fname), filesIds), total=len(filesIds)))
#     print(results)
    statistic, ids = zip(*results) # pack into Statistic and Ids 
    # create new dataframe with n statistic sensor data
#     data = pd.DataFrame(statistic, columns=[f"stat_{i}" for i in range(len(statistic[0]))])
    data = pd.DataFrame(statistic, columns=sensorStatCol)
    data["id"] = ids # add ids into dataframe
#     print(data)
    
    return data

trainTSData = loadTimeSeriesData((CFG.rootDir + CFG.trainSeriesParquet))
testTSData = loadTimeSeriesData((CFG.rootDir + CFG.testSeriesParquet))
combinedTrainDF =  pd.merge(trainDataDF, trainTSData, how="left", on='id')
combinedTestDF = pd.merge(testDataDF, testTSData, how="left", on='id')
combinedTrainDF = combinedTrainDF.drop("id", axis=1)
combinedTestDF = combinedTestDF.drop("id", axis=1)
combinedTrainDF = combinedTrainDF.dropna(subset="sii")

columnsTestCategory = []
for col in combinedTestDF.columns:
    if combinedTestDF[col].dtype == "object":
        columnsTestCategory.append(col)
columnsTestNum = []
for col in combinedTestDF.columns:
    if combinedTestDF[col].dtype != "object" and col !="sii":
        columnsTestNum.append(col)
trainCol =columnsTestCategory + columnsTestNum +["sii"]
testCol = columnsTestCategory + columnsTestNum
combinedTrainDF =combinedTrainDF[trainCol]
combinedTestDF = combinedTestDF[testCol]

# fill the Nan (missing) in for category
for c in columnsTestCategory:
    combinedTrainDF[c] = combinedTrainDF[c].fillna('missing') # fill na for missing category
    combinedTestDF[c] = combinedTestDF[c].fillna("missing") 
#  fill the Nan numberic with median value 
for c in columnsTestNum:
    combinedTrainDF[c] = combinedTrainDF[c].fillna(combinedTrainDF[c].median())
    combinedTestDF[c] = combinedTestDF[c].fillna(combinedTestDF[c].median())

# define Category dictionary for encode 
catDic = {"missing": 0, "Spring": 1, "Summer": 2, "Fall": 3, "Winter" : 4}
for c in columnsTestCategory:
    combinedTrainDF[c] = combinedTrainDF[c].map(catDic)
    combinedTestDF[c] =combinedTestDF[c].map(catDic)

combinedTrainDF['Physical-Weight'].replace(0, combinedTrainDF['Physical-Weight'].median() , 
                                           inplace=True )
combinedTrainDF["Physical-BMI"].replace(0, combinedTrainDF["Physical-BMI"].median() , inplace=True)
def getBMIInfo(df):
    """
    classify Weight category by BMI reference from World Health Organization  
    """
    # print(df)
    bmi = df["Physical-BMI"]
    age = df["Basic_Demos-Age"]
    gender = df["Basic_Demos-Sex"]
    bodyweightClass = 0
    if df["Basic_Demos-Sex"] == 0: # for female
        if age < 8 :
            adaptThreshold1 = 12.0
            adaptThreshold2 = 13.0
            adaptThreshold3 = 17.7
            adaptThreshold4 = 0.5*(age-5) + 19.0 
            if bmi < adaptThreshold1:
                bodyweightClass = 0 # Severe thinness
            elif bmi <adaptThreshold2:
                bodyweightClass = 1 # thiness
            elif bmi <adaptThreshold3:
                bodyweightClass = 2 # normal 
            elif bmi <adaptThreshold4:
                 bodyweightClass = 3 # Overweight 
            else :
                 bodyweightClass = 4 # Obesity

            return bodyweightClass
            
        elif age < 15:
            # apply linear regression 
            adaptThreshold1 = 0.357 * (age-8) + 12.0
            adaptThreshold2 = 0.429 * (age-8) + 13.0
            adaptThreshold3 = 0.828 * (age-8) + 17.7
            adaptThreshold4 = 1.1*(age-8) + 20.5 
            if bmi < adaptThreshold1:
                bodyweightClass = 0 # Severe thinness
            elif bmi <adaptThreshold2:
                bodyweightClass = 1 # thiness
            elif bmi <adaptThreshold3:
                bodyweightClass = 2 # normal 
            elif bmi <adaptThreshold4:
                 bodyweightClass = 3 # Overweight 
            else :
                 bodyweightClass = 4 # Obesity

            return bodyweightClass
        
        elif age < 19:
            # apply linear regression 
            adaptThreshold1 = 14.7
            adaptThreshold2 = 16.5
            adaptThreshold3 = 0.5 * (age-15) + 23.5
            adaptThreshold4 = 0.325 *(age-15) + 28.2 
            if bmi < adaptThreshold1:
                bodyweightClass = 0 # Severe thinness
            elif bmi <adaptThreshold2:
                bodyweightClass = 1 # thiness
            elif bmi <adaptThreshold3:
                bodyweightClass = 2 # normal 
            elif bmi <adaptThreshold4:
                 bodyweightClass = 3 # Overweight 
            else :
                 bodyweightClass = 4 # Obesity

            return bodyweightClass
        else: 
            #use adult
            if bmi < 18.5: 
                bodyweightClass = 0 # thiness 
            elif bmi < 24.9:
                bodyweightClass = 2 # noraml 
            elif bmi < 29.9:
                 bodyweightClass = 3 # Overweight 
            else:
                bodyweightClass = 4 # Obesity

            return bodyweightClass


    else: # for male
        if age < 8 :
            adaptThreshold1 = 12.5
            adaptThreshold2 = 13.2
            adaptThreshold3 = 0.266 *(age-5) + 16.7
            adaptThreshold4 = 0.5*(age-5) + 18.2 
            if bmi < adaptThreshold1:
                bodyweightClass = 0 # Severe thinness
            elif bmi <adaptThreshold2:
                bodyweightClass = 1 # thiness
            elif bmi <adaptThreshold3:
                bodyweightClass = 2 # normal 
            elif bmi <adaptThreshold4:
                 bodyweightClass = 3 # Overweight 
            else :
                 bodyweightClass = 4 # Obesity

            return bodyweightClass
            
        elif age < 15:
            # apply linear regression 
            adaptThreshold1 = 0.314 * (age-8) + 12.5
            adaptThreshold2 = 0.4 * (age-8) + 13.2
            adaptThreshold3 = 0.742 * (age-8) + 17.5
            adaptThreshold4 = 1.04 *(age-8) + 19.7 
            if bmi < adaptThreshold1:
                bodyweightClass = 0 # Severe thinness
            elif bmi <adaptThreshold2:
                bodyweightClass = 1 # thiness
            elif bmi <adaptThreshold3:
                bodyweightClass = 2 # normal 
            elif bmi <adaptThreshold4:
                 bodyweightClass = 3 # Overweight 
            else :
                 bodyweightClass = 4 # Obesity

            return bodyweightClass
        
        elif age < 19:
            # apply linear regression 
            adaptThreshold1 = 15.7
            adaptThreshold2 = 0.375 * (age-15) + 16.0
            adaptThreshold3 = 0.7 * (age-15) + 22.7
            adaptThreshold4 = 0.675 *(age-15) + 27.0
            if bmi < adaptThreshold1:
                bodyweightClass = 0 # Severe thinness
            elif bmi <adaptThreshold2:
                bodyweightClass = 1 # thiness
            elif bmi <adaptThreshold3:
                bodyweightClass = 2 # normal 
            elif bmi <adaptThreshold4:
                 bodyweightClass = 3 # Overweight 
            else :
                 bodyweightClass = 4 # Obesity

            return bodyweightClass
        else: 
            #use adult
            if bmi < 18.5: 
                bodyweightClass = 0 # thiness 
            elif bmi < 24.9:
                bodyweightClass = 2 # noraml 
            elif bmi < 29.9:
                 bodyweightClass = 3 # Overweight 
            else:
                bodyweightClass = 4 # Obesity
            return bodyweightClass
def getWaistHeightInfo(df):
    age = df["Basic_Demos-Age"]
    gender = df["Basic_Demos-Sex"]
    ratio = df["Physical-Waist_Circumference"] / df["Physical-Height"]
    ratioClass = 0
    if age < 15:
        if ratio < 0.34:
            ratioClass =0 # Extremely Slim
        elif ratio < 0.45:
            ratioClass  = 1 # slim
        elif ratio < 0.51:
            ratioClass = 2 # Healthy
        elif ratio < 0.63:
            ratioClass = 3 # over weight
        else:
            ratioClass = 5 # Obese
        return ratioClass 

    else : # for adult 
        if gender == 0: # for female
            if ratio < 0.34:
                ratioClass =0 # Extremely Slim
            elif ratio < 0.41:
                ratioClass  = 1 # slim
            elif ratio < 0.48:
                ratioClass = 2 # Healthy
            elif ratio < 0.53:
                ratioClass = 3 # over weight
            elif ratio < 0.57:
                ratioClass = 4 # Very Overweight
            else:
                ratioClass = 5 # Obese
            return ratioClass
        else : # for male
            if ratio < 0.34:
                ratioClass =0 # Extremely Slim
            elif ratio < 0.42:
                ratioClass  = 1 # slim
            elif ratio < 0.52:
                ratioClass = 2 # Healthy
            elif ratio < 0.57:
                ratioClass = 3 # over weight
            elif ratio < 0.62:
                ratioClass = 4 # Very Overweight
            else:
                ratioClass = 5 # Obese
            return ratioClass
def getBodyFatPercentInfo(df):
    """
    classify the user health status by body fat percentage
    """
    gender = df["Basic_Demos-Sex"]
    fatPercent = df["BIA-BIA_Fat"]
    fatclass =0
    if gender == 0: # for female
        if fatPercent < 13:  
            fatclass = 0 # Essential Fat
        elif fatPercent < 20:
            fatclass = 1 # Athletes
        elif fatPercent < 24:
            fatclass = 2 # Fitness
        elif fatPercent < 31:
            fatclass = 3 # acceptable 
        else:
            fatclass = 4 # acceptable 
        return fatclass
    else: # for male
        if fatPercent < 5:  
            fatclass = 0 # Essential Fat
        elif fatPercent < 13:
            fatclass = 1 # Athletes
        elif fatPercent < 17:
            fatclass = 2 # Fitness
        elif fatPercent < 24:
            fatclass = 3 # acceptable 
        else:
            fatclass = 4 # acceptable
        return fatclass
def featureEngineering(df):
    df["BMI-AGE"] = df["Physical-BMI"] * df["Basic_Demos-Age"]
    df["BMI-Classify"]= df.apply(getBMIInfo, axis=1)
    df["Waist-Height-Ratio"] = df["Physical-Waist_Circumference"] / df["Physical-Height"]
    df["Waist-Height-Ratio-Classify"] = df.apply(getWaistHeightInfo, axis=1) 
    df['Internet-Hours-Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI-Internet-Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df["FATPercent-Classify"]= df.apply(getBodyFatPercentInfo, axis= 1)
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat'] #  
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    df['BMR_Weight'] =df['BMR_Weight'].fillna(0)
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['DEE_Weight'] = df['DEE_Weight'].fillna(0)
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    # Feature 3
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    
    return df
combinedTestDF= featureEngineering(combinedTestDF)
combinedTrainDF = featureEngineering(combinedTrainDF)
trainCol = combinedTrainDF.columns.tolist()    
testCol = combinedTestDF.columns.tolist()    
xFeature = combinedTrainDF.drop("sii", axis=1)
yLabel = combinedTrainDF["sii"]
# Handling imbalance with SMOTE
if CFG.overSampling:
    smote = SMOTE()
    X_resampled, y_resampled = smote.fit_resample(xFeature, yLabel)
    print(len(X_resampled))
    xFeature = X_resampled
    yLabel = y_resampled
from sklearn.utils import compute_sample_weight, class_weight
# scale 
if CFG.normalize:
    scale = StandardScaler()
    scaledXFeature = scale.fit_transform(xFeature)
    xFeature = scaledXFeature
    scaledTestDF    = scale.fit_transform(combinedTestDF)
    combinedTestDF = scaledTestDF
if CFG.crossValidate:
    straKFold = StratifiedKFold(n_splits=CFG.nFold, random_state=42, shuffle=True)
else:
    X_train, X_test, y_train, y_test = train_test_split(xFeature, yLabel, test_size=CFG.test, 
                                                    random_state=42, stratify=yLabel)
    # random_state设置为固定值，让每次划分train、test dataset能一致，结果可重复
    # stratify 划分train、test的依据是按照yLabel的比例，避免样本不均衡在train、test之间发生
    print(X_train.shape)
    print(X_test.shape)  
    print(y_train.shape)
    print(y_test.shape)
    # Compute sample_weight using compute_sample_weight
    sampleWeight = compute_sample_weight('balanced', y_train)
    classes_weights = class_weight.compute_sample_weight(
        class_weight='balanced',
        y=y_train)
    print(classes_weights)
    print(len(sampleWeight))

100%|██████████| 2/2 [00:00<00:00,  8.94it/s]
/var/folders/bt/t9j0x49157j0mmn3fmdnq04r0000gn/T/ipykernel_42092/3038173533.py:83: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combinedTrainDF['Physical-Weight'].replace(0, combinedTrainDF['Physical-Weight'].median() ,
/var/folders/bt/t9j0x49157j0mmn3fmdnq04r0000gn/T/ipykernel_42092/3038173533.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace

6376
(5100, 173)
(1276, 173)
(5100,)
(1276,)
[1. 1. 1. ... 1. 1. 1.]
5100


In [5]:
y_train.shape

(5100,)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 假设 X_train 和 y_train 已经准备好
input_dim = X_train.shape[1]  # 输入特征数目

# 创建模型
model = Sequential([
    Dense(128, activation='relu', input_dim=input_dim),  # 第一层
    Dropout(0.4),  # 防止过拟合
    Dense(64, activation='relu'),  # 第二层
    Dense(32, activation='relu'),  # 第三层
    Dense(16, activation='relu'),  # 第四层
    Dense(5, activation='softmax')  # 输出层 (5个类别的softmax)
])

# 编译模型
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='sparse_categorical_crossentropy',  # 当label是整数，用sparse_categorical_crossentropy
              metrics=['accuracy'])

# 训练模型
history = model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.2)

Epoch 1/1000


/Users/jiazhiyuan/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2190 - loss: 2462768431104.0000 - val_accuracy: 0.2765 - val_loss: 1.5949
Epoch 2/1000
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2444 - loss: 13809750016.0000 - val_accuracy: 0.2765 - val_loss: 1.5790
Epoch 3/1000
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2478 - loss: 3041681408.0000 - val_accuracy: 0.2078 - val_loss: 1.5647
Epoch 4/1000
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2487 - loss: 692425408.0000 - val_accuracy: 0.2078 - val_loss: 1.5519
Epoch 5/1000
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2611 - loss: 777365632.0000 - val_accuracy: 0.2078 - val_loss: 1.5403
Epoch 6/1000
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2497 - loss: 176297968.0000 - val_accuracy: 0.2078 - val_loss: 1.5298
Epoch 7/1000
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2581 - loss: 41315172.0000 - val_accuracy: 0.2078 - val_loss: 1.5204
Epoch 8/1000
128/128 ━━━━━━━━━━━━━━━━━━━━

In [16]:
for epoch, (train_acc, val_acc) in enumerate(zip(history.history['accuracy'], history.history['val_accuracy']), start=1):
    print(f"Epoch {epoch}: Training Accuracy = {train_acc:.4f}, Validation Accuracy = {val_acc:.4f}")

Epoch 1: Training Accuracy = 0.2392, Validation Accuracy = 0.2069
Epoch 2: Training Accuracy = 0.2453, Validation Accuracy = 0.2088
Epoch 3: Training Accuracy = 0.2441, Validation Accuracy = 0.2088
Epoch 4: Training Accuracy = 0.2610, Validation Accuracy = 0.2088
Epoch 5: Training Accuracy = 0.2485, Validation Accuracy = 0.2098
Epoch 6: Training Accuracy = 0.2551, Validation Accuracy = 0.2098
Epoch 7: Training Accuracy = 0.2522, Validation Accuracy = 0.2127
Epoch 8: Training Accuracy = 0.2436, Validation Accuracy = 0.2088
Epoch 9: Training Accuracy = 0.2444, Validation Accuracy = 0.2098
Epoch 10: Training Accuracy = 0.2458, Validation Accuracy = 0.2088
Epoch 11: Training Accuracy = 0.2608, Validation Accuracy = 0.2618
Epoch 12: Training Accuracy = 0.2414, Validation Accuracy = 0.2657
Epoch 13: Training Accuracy = 0.2453, Validation Accuracy = 0.2686
Epoch 14: Training Accuracy = 0.2500, Validation Accuracy = 0.2657
Epoch 15: Training Accuracy = 0.2485, Validation Accuracy = 0.2696
Epoc